In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("traffic-density-w6zd9").project("traffic-density-izacs-uhqed")
version = project.version(3)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-Density-3 in yolov8:: 100%|██████████| 33418/33418 [00:04<00:00, 6845.72it/s]


In [2]:
!pip install ultralytics
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.6 MB/s eta 0:00:00


In [3]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO

# SpikeNet Implementation
class SpikeConv(nn.Module):
    """Spiking Convolutional Layer with Membrane Potential"""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dropout=0.5):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.dropout = nn.Dropout2d(dropout)
        self.membrane = nn.Parameter(torch.zeros(1, out_channels, 1, 1))
        self.threshold = 1.0
        
    def forward(self, x):
        mem_potential = self.membrane.repeat(x.size(0), 1, x.size(2), x.size(3))
        conv_out = self.conv(x)
        mem_potential = mem_potential + conv_out
        spike = (mem_potential >= self.threshold).float()
        mem_potential = mem_potential * (1 - spike)
        return self.dropout(spike)

class SpikeNet(nn.Module):
    """SpikeNet Neck Architecture"""
    def __init__(self, in_channels=[256, 512, 1024], out_channels=[256, 512, 1024], dropout=0.5):
        super().__init__()
        
        # Bottom-up pathway
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        # Lateral connections
        self.lateral_conv1 = SpikeConv(in_channels[2], out_channels[2], 1, dropout=dropout)
        self.lateral_conv2 = SpikeConv(in_channels[1], out_channels[1], 1, dropout=dropout)
        
        # Fusion layers
        self.fusion_conv1 = SpikeConv(out_channels[2], out_channels[2], 3, padding=1, dropout=dropout)
        self.fusion_conv2 = SpikeConv(out_channels[1], out_channels[1], 3, padding=1, dropout=dropout)
        self.fusion_conv3 = SpikeConv(out_channels[0], out_channels[0], 3, padding=1, dropout=dropout)

    def forward(self, features):
        c3, c4, c5 = features  # Features from backbone
        
        # Top-down pathway
        p5 = self.lateral_conv1(c5)
        p4 = self.lateral_conv2(c4) + F.interpolate(p5, scale_factor=2)
        p3 = self.downsample(c3) + F.interpolate(p4, scale_factor=2)
        
        # Bottom-up pathway
        n3 = self.fusion_conv1(p3)
        n4 = self.fusion_conv2(p4) + self.upsample(n3)
        n5 = self.fusion_conv3(p5) + self.upsample(n4)
        
        return [n3, n4, n5]

# YOLOv8 + SpikeNet Integration
class YOLOv8SpikeNet(YOLO):
    """Custom YOLOv8 with SpikeNet Neck"""
    def __init__(self, model='yolov8n.yaml', dropout=0.5):
        super().__init__(model)
        self.replace_neck(dropout)
    
    def replace_neck(self, dropout):
        # Replace PANet with SpikeNet in model architecture
        self.model.model[-1] = SpikeNet(
            in_channels=[256, 512, 1024],
            out_channels=[256, 512, 1024],
            dropout=dropout
        )

# Training Configuration
if __name__ == '__main__':
    # Initialize custom YOLOv8 + SpikeNet model
    model = YOLOv8SpikeNet(model='yolov8n.yaml')
    
    # Train the model
    results = model.train(
        data='/kaggle/working/Traffic-Density-3/data.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="AdamW",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.5
    )

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/Traffic-Density-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=Fal

100%|██████████| 755k/755k [00:00<00:00, 22.7MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 105MB/s]


AMP: checks passed ✅


...


50 epochs completed in 2.430 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:17<00:00,  3.17it/s]


                   all       1811       9377      0.641      0.716      0.738      0.459
               Bicycle        398        511      0.656      0.933      0.918      0.522
                E-bike        504        516      0.833      0.973      0.981       0.67
               Jeepney        837       1592      0.631      0.592      0.677      0.402
            Motorcycle        694       1556      0.449      0.436       0.45      0.214
            Pedestrian        568       1449      0.572      0.326      0.457      0.211
              Tricycle        684        882       0.85      0.868      0.911      0.604
                 Truck        728        882      0.738      0.881      0.878       0.61
               Vehicle        494       1989      0.399      0.719      0.633      0.438


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train
